## Handling Missing and Incorrect Data

In an organization, data comes from so many different sources following different standards and different quality control across the globe. While working with the data, many times, we see missing or incorrect data value in a particular field. For example –
“ “ (space) in an amount field , “$”, “?”, “aa”, “,” in amount field (amount field should have numeric for mathematical calculations).

Most of these errors are due to human data entry error or intentionally not updating the data, equipment error, faulty measurements, incorrect ETL (Extraction, Transformation and Loading) process and other faulty processes. Faulty or missing data can lead to incorrect and incompetent calculations leading to bias and underfitting a model.

Data can be completely missing at random (due to reasons explained above) or could be due to optional fields in the forms (for example, cell number is mandatory, home number is optional) or Survey questions that can be skipped or any field that can be skipped.

So before handling a machine learning problem, it becomes utmost important to clean, standardize and prepare the data so that it can be used effectively for the building efficient machine learning models. Preparing the data takes majority of time for a data scientist and data engineers (anywhere around 50% to 70% of total time).

In this tutorial we will learn about how to identify the missing data or incorrect data and how to replace the same. We will be using Python code to illustrate and solve.

In [1]:
#Import Library for Data manipulation
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer

#Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
dataset = pd.read_csv("C://Users//jagan//OneDrive//Documents//jagannathbanerjee.com//Blog//Height_Weight_Data.csv")

### Step 1 : Get the shape of the dataset (rows, columns) 

In [3]:
dataset.shape

(200, 3)

#### Observation - we have 200 rows and 3 columns.

### Step 2 : Get the column names

In [4]:
list(dataset.columns)

['Index', 'Height(Inches)', 'Weight(Pounds)']

#### We have 3 columns 'Index', 'Height(Inches)', 'Weight(Pounds)' . Always a good practice to remove brackets and replace with underscores.

In [5]:
# Renaming the columns using pandas rename function.
dataset = dataset.rename(columns={'Height(Inches)':'Height_in_Inches', 'Weight(Pounds)' : 'Weight_in_Pounds'})

#getting the column names again
list(dataset.columns)

['Index', 'Height_in_Inches', 'Weight_in_Pounds']

### Step 3 : Get the column information or Datatype

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
Index               200 non-null int64
Height_in_Inches    197 non-null float64
Weight_in_Pounds    198 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 4.8+ KB


#### Lookout!!!! 
Weight_in_Pounds is non-numeric. This raises  red flag. We are expecting weight to be a numeric field but it turns out to be object or non-numeric. We will assess this field more

### Step 4 : Getting basic statistics for the columns

In [7]:
dataset.describe()  # Basic Statistics on Numeric Columns

,Index,Height_in_Inches
count,200.000000,197.000000
mean,100.500000,67.590508
std,57.879185,5.211051
min,1.000000,0.000000
25%,50.750000,66.500000
50%,100.500000,67.930000
75%,150.250000,69.200000
max,200.000000,73.900000


#### Observation for Height_in_Inches Column:
1. Look at the row - "count" . Index has 200 rows while Height_in_Inches contains 197 rows. There must be 3 rows missing data
2. Look at the row "min" - Height_in_Inches contains 0 value. Minimum height is 0, which is impossible. This must an error.

In [8]:
dataset.describe(include=['O'])  # Frequency table for non numeric columns

,Weight_in_Pounds
count,198
unique,194
top,141.49
freq,2


#### Observation for Weight_in_Pounds Column:
1. Look at the row - "count" . It contains 198 rows. 2 rows missing data
2. Look at the row "unique" - It has 194 unique counts, meaning rest 6 rows must be having something different. We will find it out.

### Step 5 : Handling Height_in_Inches Column

In [9]:
#Identifying the rows containing missing data
missing_value_row = list(dataset[dataset['Height_in_Inches'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))

Missing Value Rows--> [12, 110, 187] 
Missing data in rows--> 3


#### Observation:
Row 12, 110, 187 contains missing data

### Step 5a : Filling the missing values
We can fill the missing values with different options depending on the situation. We can fill with
1. Mean
2. Median
3. Calculated Value
4. Hard-Coded Value

In [10]:
# Median of all Heights can be found using the below function
height_median = dataset['Height_in_Inches'].median()
height_median

67.93

In [11]:
# Mean of all Heights can be found using the below function
Height_mean = dataset['Height_in_Inches'].mean()
Height_mean

67.59050761421318

#### Step 5a.1 Filling missing value with Median

In [12]:
# We will copy the data into a new datset and replace the values with median .
dataset_fix_with_med = dataset.copy()
#Replacing missing data with Tenure X MonthlyCharges
for missing_row in missing_value_row :
    dataset_fix_with_med['Height_in_Inches'][missing_row] = height_median

In [13]:
#Checking the values in missing rows 12, 110, 187:
print(dataset_fix_with_med['Height_in_Inches'][12])
print(dataset_fix_with_med['Height_in_Inches'][110])
print(dataset_fix_with_med['Height_in_Inches'][187])

67.93
67.93
67.93


#### Step 5a.2 Filling missing value with Mean

In [14]:
# We will copy the data into a new datset and replace the values with mean .
dataset_fix_with_mean = dataset.copy()
#Replacing missing data with Tenure X MonthlyCharges
for missing_row in missing_value_row :
    dataset_fix_with_mean['Height_in_Inches'][missing_row] = Height_mean

In [15]:
#Checking the values in missing rows 12, 110, 187:
print(dataset_fix_with_mean['Height_in_Inches'][12])
print(dataset_fix_with_mean['Height_in_Inches'][110])
print(dataset_fix_with_mean['Height_in_Inches'][187])

67.59050761421318
67.59050761421318
67.59050761421318


#### Step 5a.3 Filling missing value with Calculated Value

In [16]:
# We will copy the data into a new datset  .
dataset_fix_with_Calcv = dataset.copy()

#Replacing missing data with calculated value. Height field will have weight/1.5 as calculated value
for missing_row in missing_value_row :
    dataset_fix_with_Calcv['Height_in_Inches'][missing_row] = float(dataset_fix_with_Calcv['Weight_in_Pounds'][missing_row])/1.5

#Checking the values in missing rows 12, 110, 187:
print(dataset_fix_with_Calcv['Height_in_Inches'][12])
print(dataset_fix_with_Calcv['Height_in_Inches'][110])
print(dataset_fix_with_Calcv['Height_in_Inches'][187])

83.74
81.37333333333333
80.61333333333333


In [17]:
dataset_fix_with_med.describe()
#dataset_fix_with_mean.describe()
#dataset_fix_with_Calcv.describe()

,Index,Height_in_Inches
count,200.000000,200.000000
mean,100.500000,67.595600
std,57.879185,5.171788
min,1.000000,0.000000
25%,50.750000,66.500000
50%,100.500000,67.930000
75%,150.250000,69.200000
max,200.000000,73.900000


#### Observation
We have fixed the height_in_inches . Now count is same in as in index, which means missing value is taken care of. Next we have to fix is the min field which is 0. Height cannot be 0. We will identifying the rows containing 0 and replace them with value in 25% percentile.

In [18]:
zero_value_row = list(dataset_fix_with_med[dataset_fix_with_med['Height_in_Inches']==0].index)
zero_value_row

[158]

In [19]:
# So there is only 1 row with zero value. We will replace that with 25th percentile value
dataset_fix_with_med['Height_in_Inches'].quantile(0.25)

#Replacing the row 158 with 25th percentile value
dataset_fix_with_med['Height_in_Inches'][158] = dataset_fix_with_med['Height_in_Inches'].quantile(0.25)

In [20]:
dataset_fix_with_med.describe()

,Index,Height_in_Inches
count,200.000000,200.00000
mean,100.500000,67.92810
std,57.879185,1.91877
min,1.000000,63.43000
25%,50.750000,66.50000
50%,100.500000,67.93000
75%,150.250000,69.20000
max,200.000000,73.90000


#### Observation
So No missing values and minimum value 0 replaced with 25th percentile

### Step 5b : Deleting the missing value rows

In [21]:
# Making a copy of original dataset
dataset_fix_with_del = dataset.copy()

#Identifying the rows containing missing data
missing_value_row = list(dataset_fix_with_del[dataset_fix_with_del['Height_in_Inches'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))

Missing Value Rows--> [12, 110, 187] 
Missing data in rows--> 3


In [22]:
# Dropping the rows with empty values
dataset_fix_with_del = dataset_fix_with_del.dropna()
dataset_fix_with_del.describe()

,Index,Height_in_Inches
count,195.000000,195.000000
mean,100.179487,67.605436
std,57.819249,5.234161
min,1.000000,0.000000
25%,50.500000,66.500000
50%,99.000000,67.940000
75%,149.500000,69.205000
max,200.000000,73.900000


#### Observation 
Although we had 3 empty rows for height, it deleted 5 rows since it found 2 empty rows in the weight column.
Beware while using dropna function.

In [23]:
# Making a copy of original dataset
dataset_fix_with_del = dataset.copy()

dataset_fix_with_del = dataset_fix_with_del.dropna(subset=['Height_in_Inches'])
dataset_fix_with_del.describe()

,Index,Height_in_Inches
count,197.000000,197.000000
mean,100.446701,67.590508
std,57.641870,5.211051
min,1.000000,0.000000
25%,51.000000,66.500000
50%,100.000000,67.930000
75%,150.000000,69.200000
max,200.000000,73.900000


#### Observation 
3 rows with values in height column got deleted.

In [25]:
#Other Flavours 
    
#drop only if ALL columns are NaN
dataset_fix_with_del.dropna(how='all')

#Drop row if it does not have at least two values that are **not** NaN
dataset_fix_with_del.dropna(thresh=2)   

#More can be found at : https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html


,Index,Height_in_Inches,Weight_in_Pounds
0,1,65.78,112.99
1,2,71.52,136.49
2,3,69.40,153.03
3,4,68.22,142.34
4,5,67.79,144.3
5,6,68.70,123.3
6,7,69.80,141.49
7,8,70.01,136.46
8,9,67.90,112.37
9,10,66.78,120.67


## Replacing missing data with scikit learn Imputer

In [26]:
#Copying the height data into a dataframe
height_data = pd.DataFrame({'Height_in_Inches' : dataset['Height_in_Inches']})
print(height_data.head())

#Identifying the rows containing missing data
missing_value_row = list(height_data[height_data['Height_in_Inches'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))
      
#Finding mean & Median
print( 'Mean:', height_data['Height_in_Inches'].mean())
print( 'Median:', height_data['Height_in_Inches'].median())

   Height_in_Inches
0             65.78
1             71.52
2             69.40
3             68.22
4             67.79
Missing Value Rows--> [12, 110, 187] 
Missing data in rows--> 3
Mean: 67.59050761421318
Median: 67.93


In [27]:
# NaN represents blank values, it will be replaced with mean across the column.
# Use strategy = 'median' if you want to replace missing value with median.
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)  

# Fit the imputer on the height column.
height_data['Height_in_Inches']= imputer.fit_transform(height_data[['Height_in_Inches']])

#Identifying the rows containing missing data
missing_value_row = list(height_data[height_data['Height_in_Inches'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))

#Verify of the missing rows was replaced with mean
print(height_data['Height_in_Inches'][12])
print(height_data['Height_in_Inches'][110])
print(height_data['Height_in_Inches'][187])

Missing Value Rows--> [] 
Missing data in rows--> 0
67.5905076142132
67.5905076142132
67.5905076142132


In [28]:
height_data.describe()

,Height_in_Inches
count,200.000000
mean,67.590508
std,5.171622
min,0.000000
25%,66.500000
50%,67.865000
75%,69.200000
max,73.900000


In [29]:
# Height cannot be 0 . Let's replace the 0 value with median. First Step, we have to find the 0 value row
#Identifying the 0 value rows
zero_value_row = list(height_data[height_data['Height_in_Inches'] == 0].index)
print('Zero Value Rows-->', zero_value_row)

Zero Value Rows--> [158]


In [30]:
# Row 167 has 0 value. We will replace the 0 value with median. Weight cannot be 0.
height_data['Height_in_Inches'][zero_value_row] = height_data['Height_in_Inches'].median()

In [31]:
#Verify the replacement
height_data['Height_in_Inches'][zero_value_row]

158    67.865
Name: Height_in_Inches, dtype: float64

In [32]:
height_data.describe()

,Height_in_Inches
count,200.000000
mean,67.929833
std,1.916551
min,63.430000
25%,66.522500
50%,67.882500
75%,69.200000
max,73.900000


### Looks all set! All data cleaned.

## Fix Weight in Pounds Field

In [33]:
#Creating a dataset for weight.
weight_data = pd.DataFrame({'Weight_in_Pounds' : dataset['Weight_in_Pounds']})

#printing first 5 rows
print("First 5 rows")
print(weight_data.head())

# Basic variable info
print('\nVariable information - ')
print(weight_data.info())

#Getting the statistics
print("\nGetting Basic Statistics\n")
print(weight_data.describe(include=['O']))

First 5 rows
  Weight_in_Pounds
0           112.99
1           136.49
2           153.03
3           142.34
4            144.3

Variable information - 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 1 columns):
Weight_in_Pounds    198 non-null object
dtypes: object(1)
memory usage: 1.6+ KB
None

Getting Basic Statistics

       Weight_in_Pounds
count               198
unique              194
top              141.49
freq                  2


### Observation
1. Weight_in_Pounds is non-numeric. This raises red flag. We are expecting weight to be a numeric field but it turns out to be object or non-numeric. We will assess this field more
2. Look at the row - "count" . It contains 198 rows. 2 rows missing data
3. Look at the row "unique" - It has 194 unique counts, meaning rest 6 rows must be having something different. We will find it out.

In [34]:
# Printing missing row and data
print(weight_data[pd.to_numeric(weight_data['Weight_in_Pounds'], errors='coerce').isnull()])

#Capturing missing rows into  a list
rows = list(weight_data[pd.to_numeric(weight_data['Weight_in_Pounds'], errors='coerce').isnull()].index)
print('Rows with incorrect data ->', rows)

    Weight_in_Pounds
100              NaN
116                ?
128                 
151              NaN
Rows with incorrect data -> [100, 116, 128, 151]


In [35]:
#We will replace will rows NaN/ Not NaN or any garbage data with NaN 
for row_num in rows :
    weight_data['Weight_in_Pounds'][row_num] = np.nan

In [36]:
#Identifying the rows containing missing data
missing_value_row = list(weight_data[weight_data['Weight_in_Pounds'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))

Missing Value Rows--> [100, 116, 128, 151] 
Missing data in rows--> 4


In [37]:
weight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 1 columns):
Weight_in_Pounds    196 non-null object
dtypes: object(1)
memory usage: 1.6+ KB


In [38]:
# We will convert the datatype of weight column from object to float. Thi is needed for imputer replacement.
weight_data['Weight_in_Pounds'] = weight_data['Weight_in_Pounds'].astype(float)

In [39]:
#We wil replace the missing weight rows with median value
imputer = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)  

# Fit the imputer on the height column.
weight_data['Weight_in_Pounds']= imputer.fit_transform(weight_data[['Weight_in_Pounds']])

#Identifying the rows containing missing data
missing_value_row = list(weight_data[weight_data['Weight_in_Pounds'].isnull()].index)
print('Missing Value Rows-->', missing_value_row , '\nMissing data in rows-->', len(missing_value_row))

#Verify of the missing rows was replaced with mean
print(weight_data['Weight_in_Pounds'][100])
print(weight_data['Weight_in_Pounds'][116])
print(weight_data['Weight_in_Pounds'][128])
print(weight_data['Weight_in_Pounds'][151])

Missing Value Rows--> [] 
Missing data in rows--> 0
127.76
127.76
127.76
127.76


### Finally now we have converted the datatype to float and replaced missing values with median. 
### Next we will again use describe() to evallute the field

In [40]:
weight_data.describe()

,Weight_in_Pounds
count,200.000000
mean,126.585750
std,14.803948
min,0.000000
25%,119.895000
50%,127.760000
75%,135.480000
max,158.960000


#### Minimum is Still 0. We will replace the 0 value with median

In [41]:
#Identifying the 0 value rows
zero_value_row = list(weight_data[weight_data['Weight_in_Pounds'] == 0].index)
print('Zero Value Rows-->', zero_value_row)

Zero Value Rows--> [167]


In [42]:
# Row 167 has 0 value. We will replace the 0 value with median. Weight cannot be 0.
weight_data['Weight_in_Pounds'][zero_value_row] = weight_data['Weight_in_Pounds'].median()

In [43]:
#Verify the replacement
weight_data['Weight_in_Pounds'][zero_value_row]

167    127.76
Name: Weight_in_Pounds, dtype: float64

## Perfect! Now we will create the final  dataframe with clean and fixed data

In [44]:
clean_data = pd.DataFrame({'Height_in_Inches' : height_data['Height_in_Inches'],'Weight_in_Pounds' : weight_data['Weight_in_Pounds'] })

In [45]:
clean_data.head()

,Height_in_Inches,Weight_in_Pounds
0,65.78,112.99
1,71.52,136.49
2,69.40,153.03
3,68.22,142.34
4,67.79,144.30


In [46]:
clean_data.describe()

,Height_in_Inches,Weight_in_Pounds
count,200.000000,200.000000
mean,67.929833,127.224550
std,1.916551,11.757181
min,63.430000,97.900000
25%,66.522500,119.997500
50%,67.882500,127.760000
75%,69.200000,135.480000
max,73.900000,158.960000


## Data looks pretty clean. Kudos!